In [1]:
# Modifying the provided 'edited-model.py' to incorporate suggested improvements

# Modified script content with additional tuning, early stopping, and class distribution checks.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import joblib

In [2]:
# Load the data
df = pd.read_csv('C:/Users/honey/Documents/placment work/Electronical-AI-Device-Recogniser/khanya/data managment/datasets/acs-f2-dataset.csv')

In [3]:
# Data Preprocessing
df = df.drop('time', axis=1)

# Convert categorical labels to numeric
le = LabelEncoder()
df['equipment'] = le.fit_transform(df['equipment'])

# Split features and labels
X = df.drop('equipment', axis=1)
y = df['equipment']

# Analyze class distribution before SMOTE
print("Class distribution before SMOTE:")
print(y.value_counts())

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance using SMOTE
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_scaled, y)

Class distribution before SMOTE:
equipment
3     9682
5     9388
7     9378
6     9348
2     9272
14    9155
13    9088
8     9015
11    9004
4     8849
12    8810
9     8784
10    8755
1     8688
0     8661
Name: count, dtype: int64


In [4]:
# Analyze class distribution after SMOTE
print("Class distribution after SMOTE:")
print(pd.Series(y_resampled).value_counts())

# One-hot encode the target labels
num_classes = len(np.unique(y))
y_resampled_onehot = to_categorical(y_resampled, num_classes=num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled_onehot, test_size=0.2, random_state=42)

Class distribution after SMOTE:
equipment
0     9682
1     9682
2     9682
3     9682
4     9682
5     9682
6     9682
7     9682
8     9682
9     9682
10    9682
11    9682
12    9682
13    9682
14    9682
Name: count, dtype: int64


In [ ]:
# Define the model architecture with dropout
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),  # Increased dropout for stronger regularization
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),  # Added another dense layer with fewer neurons
    Dense(num_classes, activation='softmax')
])

# Compile the model with a lower learning rate for better generalization
model.compile(optimizer=Adam(learning_rate=0.0005), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])







Epoch 1/100


c:\Users\honey\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3631/3631 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.3037 - loss: 2.0550 - val_accuracy: 0.4657 - val_loss: 1.5218
Epoch 2/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4414 - loss: 1.5734 - val_accuracy: 0.5328 - val_loss: 1.3311
Epoch 3/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.4947 - loss: 1.4051 - val_accuracy: 0.5698 - val_loss: 1.2115
Epoch 4/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.5260 - loss: 1.3111 - val_accuracy: 0.6015 - val_loss: 1.1179
Epoch 5/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.5555 - loss: 1.2295 - val_accuracy: 0.6262 - val_loss: 1.0567
Epoch 6/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.5795 - loss: 1.1727 - val_accuracy: 0.6481 - val_loss: 1.0050
Epoch 7/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.5920 - loss: 1.1258 - val_accuracy: 0.6619 - val_loss: 0.9674
Epoch 8/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.6055 - loss: 1.0942 

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_true_classes[:len(y_pred_classes)], y_pred_classes)
plt.figure(figsize=(20, 8))

# Plot the regular confusion matrix
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Plot the normalized confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.subplot(1, 2, 2)
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

plt.tight_layout()
plt.show()


In [ ]:
# Save the trained model and the scaler for future use
model.save('new-model1.0.1.h5')
joblib.dump(scaler, 'new-scaler1.0.1.pkl')

In [15]:
  # Return the new path so the user can download it